In [ ]:
!pip install transformers
!pip install pyTelegramBotAPI
!pip install torch
!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.9.0-py3-none-any.whl size=203107 sha256=5b9ee0fe5cd1cdc8bd72ec036a0e954f007fc9d83f893c937be44bc94b481051
  Stored in directory: /root/.cache/pip/wheels/95/a7/fd/f667b71e29e3f64f239f30195efc39de642e72f8a47a6fcd8d
Successfully built pyTelegramBotAPI
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://

In [ ]:
!pip install transformers
!pip install pyTelegramBotAPI
!pip install torch
!pip install accelerate

import telebot
import transformers
import torch
import time
import pickle

# Inicializar el bot de Telegram
bot = telebot.TeleBot('5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM')

# Cargar el modelo BERT pre-entrenado
model = transformers.BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Cargar el diccionario de preguntas y respuestas
try:
    with open('qa_dict.pickle', 'rb') as handle:
        qa_dict = pickle.load(handle)
except:
    qa_dict = {}

# Manejar mensajes de pregunta de Telegram
@bot.message_handler(func=lambda message: True)
def answer_question(message):

    if message.text == '/start':
        bot.reply_to(message, 'Hola, mi nombre es Serina. Soy un modelo de reconocimiento del lenguaje natural.')
        bot.send_message(message.chat.id, 'Estos son ejemplos de algunas de las opciones que puedes realizar: \n /Cita_de_asesoria \n /help')
        bot.send_message(message.chat.id, 'Sabiendo esto, dime en que te puedo ayudar?')
        return
    
    if message.text == '/help':
        #muestra la lista de comandos
        bot.reply_to(message, 'Lista de comandos: \n /start \n /help \n /Comprar \n /Contacto')
        return
    
    if message.text == '/Cita_de_asesoria':
        bot.reply_to(message, 'En esta opcion podras realizar realizarme preguntas y te ayudaré a aclarar tus dudas')
        return

    if message.text == '/Contacto':
        bot.reply_to(message, 'Creado por: ')
        return
    
    # Extraer la pregunta del mensaje
    question = message.text

    # Buscar en el diccionario si la pregunta ya ha sido respondida anteriormente
    if question in qa_dict:
        answer = qa_dict[question]
    else:
        #extraer texto de un archivo de texto
        with open('texto.txt', 'r') as file:
            text = file.read().replace('\n', '')
            
        try:
            inputs = tokenizer.encode_plus(question, text, return_tensors='pt', add_special_tokens=True)
            output = model(**inputs)
        except Exception as e:
            bot.reply_to(message, 'Lo siento, ha ocurrido un error al procesar tu pregunta')
            return

        # Obtener la respuesta
        answer_start_scores, answer_end_scores = output[:2]
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end],skip_special_tokens=True)

        #excepcion de errores
        if answer == '[CLS]' or answer == '[SEP]':
            answer = 'No se pudo encontrar una respuesta'
        

        # Almacenar la pregunta y la respuesta en el historial
        qa_dict[question] = answer
        with open("qa_dict.pickle", "wb") as f:
            pickle.dump(qa_dict, f)

    # Enviar la respuesta al usuario
    bot.reply_to(message, answer)


# Ejecutar el bot
while True:
    try:
        bot.polling()
    except Exception:
        time.sleep(15)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.10.0-py3-none-any.whl size=205420 sha256=7eb9e23c63bfc75057721051d164fcdf1b86ccadbc34d374883c1ab471ed1992
  Stored in directory: /root/.cache/pip/wheels/ee/e3/f1/4c142d3be2e5f09c1ba8bf5801ae61d7b2cfd9afe497872806
Successfully built pyTelegramBotAPI
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https:/

2023-02-24 15:03:34,673 (__init__.py:1083 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2023-02-24 15:03:34,685 (__init__.py:1085 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/telebot/__init__.py", line 1073, in __threaded_polling
    polling_thread.raise_exceptions()
  File "/usr/local/lib/python3.8/dist-packages/telebot/util.py", line 109, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.8/dist-packages/telebot/util.py", line 91, in run
    task(*args, **kwargs)
  Fil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install transformers
#!pip install pyTelegramBotAPI
#!pip install torch
#!pip install accelerate

import telebot
import transformers
import torch
import time
import pickle

# Inicializar el bot de Telegram
bot = telebot.TeleBot('5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM')

# Cargar el modelo BERT pre-entrenado
model = transformers.BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Cargar el diccionario de preguntas y respuestas
try:
    with open('qa_dict.pickle', 'rb') as handle:
        qa_dict = pickle.load(handle)
except:
    qa_dict = {}

def select_shipping_plan(message):
    if message.text == 'Envio gratis' or message.text == 'envio gratis':
        bot.reply_to(message, 'Escogiste el plan de envio gratis')
        return
    elif message.text == 'Envio rapido' or message.text == 'envio rapido':
        bot.reply_to(message, 'Escogiste el plan de envio rapido')
        return
    elif message.text == 'Envio express' or message.text == 'envio express':
        bot.reply_to(message, 'Escogiste el plan de envio express')
        return

def select_consult(message):
    if message.text == 'Consulta de horarios' or message.text == 'consulta de horarios':
        bot.reply_to(message, 'Escogiste la opcion de consultar horarios')
        bot.send_message(message.chat.id, 'Estos son nuestros horarios de atencion:')
        bot.send_photo(message.chat.id, open('imagen1.jpg', 'rb'))
        return
    elif message.text == 'Consulta de productos' or message.text == 'consulta de productos':
        bot.reply_to(message, 'Escogiste la opcion de consultar productos')
        bot.send_message(message.chat.id, 'Enviame una imagen del producto que deseas consultar')
        # aqui entra el modelo de reconocimiento de imagenes
        return
    elif message.text == 'Destinos de envio' or message.text == 'destinos de envio':
        bot.reply_to(message, 'Escogiste la opcion de consultar destinos de envio')
        bot.send_message(message.chat.id, 'Estos son los destinos de envio:')
        bot.send_photo(message.chat.id, open('imagen2.jpg', 'rb'))
        return

def price(message):
    if message.text == 'Cotizacion de envio' or message.text == 'cotizacion de envio':
        bot.reply_to(message, 'Escogiste la opcion de cotizar envio')
        bot.send_message(message.chat.id, 'Existen 3 opciones de envio: \n \n * Envio gratis \n * Envio rapido \n * Envio express')
        bot.send_message(message.chat.id, 'La opcion gratis tiene un tiempo de entrega de 3 a 5 dias habiles, la opcion rapida tiene un tiempo de entrega de 2 a 3 dias habiles y la opcion express tiene un tiempo de entrega de 1 a 2 dias habiles')
        #espera 5 segundos
        time.sleep(20)
        bot.send_message(message.chat.id, 'Te gustaria escoger alguno de estos planes? y proceder con el envio? \n \n * Si \n * No')
    elif message.text == 'Personalizado' or message.text == 'personalizado':
        bot.reply_to(message, 'Escogiste el plan personalizado')
        return
    return


# Manejar mensajes de pregunta de Telegram
@bot.message_handler(func=lambda message: True)
def answer_question(message):

    if message.text == '/start':
        bot.reply_to(message, 'Hola, mi nombre es Serina. Soy un modelo de reconocimiento del lenguaje natural.')
        bot.send_message(message.chat.id, 'Estos son ejemplos de algunas de las opciones que puedes realizar: \n \n /Asesoria \n /Envio \n /Cotizacion \n')
        bot.send_message(message.chat.id, 'Tambien puedes consultar las opciones en el menu de la ezquina inferior izquierda o con el comando /help en todo momento')
        bot.send_message(message.chat.id, 'Sabiendo esto, dime en que te puedo ayudar?')
        return
    
    if message.text == '/Asesoria':
        bot.reply_to(message, 'Estoy aqui para orientarte escoge una opcion o dime en que te puedo ayudar: \n \n * Consulta de horarios \n * Consulta de productos \n * Destinos de envio')
        return
    else:
        select_consult(message)
    
    if message.text == '/Envio':
        bot.reply_to(message, 'Escoge un plan de envio: \n \n * Envio gratis \n * Envio rapido \n * Envio express')
        return
    else:
        select_shipping_plan(message)

    if message.text == '/Cotizacion':
        bot.reply_to(message, 'Escoge una opcion: \n \n * Cotizacion de envio \n * Cotizacion de producto')
        return
    else:
        price(message)
    

    #detener el bot
    if message.text == '/stop':
        bot.reply_to(message, 'Rutinas detenidas')
        return

    if message.text == '/help':
        #muestra la lista de comandos
        bot.reply_to(message, 'Lista de comandos: \n \n /start \n /Asesoria \n /Envio \n /Cotizacion \n /stop \n /restart \n /help')
        return

    #reiiciar el bot
    if message.text == '/restart':
        bot.reply_to(message, 'Rutinas reiniciadas')
        return
    
    
    # Extraer la pregunta del mensaje
    question = message.text

    # Buscar en el diccionario si la pregunta ya ha sido respondida anteriormente
    if question in qa_dict:
        answer = qa_dict[question]
    else:
        #extraer texto de un archivo de texto
        with open('texto.txt', 'r') as file:
            text = file.read().replace('\n', '')
            
        try:
            inputs = tokenizer.encode_plus(question, text, return_tensors='pt', add_special_tokens=True)
            output = model(**inputs)
        except Exception as e:
            bot.reply_to(message, 'Lo siento, ha ocurrido un error al procesar tu pregunta')
            return

        # Obtener la respuesta
        answer_start_scores, answer_end_scores = output[:2]
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end],skip_special_tokens=True)

        #excepcion de errores
        if answer == '[CLS]' or answer == '[SEP]':
            answer = 'No se pudo encontrar una respuesta'
        

        # Almacenar la pregunta y la respuesta en el historial
        qa_dict[question] = answer
        with open("qa_dict.pickle", "wb") as f:
            pickle.dump(qa_dict, f)

    # Enviar la respuesta al usuario
    bot.reply_to(message, answer)


# Ejecutar el bot
while True:
    try:
        bot.polling()
    except Exception:
        time.sleep(15)
        

2023-01-25 06:29:15,120 (__init__.py:1083 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty
2023-01-25 06:29:15,129 (__init__.py:1085 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/telebot/__init__.py", line 1074, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr/local/lib/python3.8/dist-packages/telebot/util.py", line 156, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.8/dist-packages/telebot/util.py", line 100, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/telebot/__init__.py", line 6395, in _run_middlewares_and_handler
    result = han

In [ ]:
import os
from telebot import TeleBot
import numpy as np
from scipy.spatial import distance
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image

# Load the ResNet50 model
model = ResNet50(weights='imagenet')

def image_comparison(img_path):
    # Load the image you want to compare
    img_to_compare = image.load_img(img_path, target_size=(224, 224))
    img_to_compare = image.img_to_array(img_to_compare)
    img_to_compare = np.expand_dims(img_to_compare, axis=0)
    img_to_compare = preprocess_input(img_to_compare)
    # Extract feature vector for the image
    feature_vec_to_compare = model.predict(img_to_compare)
    # Iterate over all images in the folder
    folder_path = "/content/data"
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            # Load and preprocess the image
            img = image.load_img(os.path.join(folder_path, filename), target_size=(224, 224))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)
            # Extract feature vector for the image
            feature_vec = model.predict(img)
            # Compute distance between the feature vectors
            dist = distance.euclidean(feature_vec_to_compare, feature_vec)
            if dist == 0:
                return "Existe el producto"
                break
            else:
                return "no tenemos de ese producto"

def handle_image(message):
    file_id = message.photo[-1].file_id
    file = bot.get_file(file_id)
    file.download('user_image.jpg')
    result = image_comparison('user_image.jpg')
    bot.send_message(message.chat.id, result)

if __name__ == '__main__':
    # Token del bot
    bot = TeleBot("5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM")
    bot.message_handler(content_types=["photo"])(handle_image)
    bot.polling()
    bot.infinity_polling()

AttributeError: ignored

In [ ]:
!pip install tensorflow
!pip install keras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install numpy
!pip install scipy
!pip install tensorflow
!pip install tensorflow-gpu
!pip install telegram


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subp

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from scipy.spatial import distance
import os

# Load the ResNet50 model
model = ResNet50(weights='imagenet')


#Get the last image in the folder to make it the one to compare to
datapath = "/content/data"
files = os.listdir(datapath)
files.sort(key=lambda x: os.path.getmtime(os.path.join(datapath, x)))
last_image = files[-1]

# Load the image you want to compare
img_to_compare = load_img("/content/data2/img2.jpg", target_size=(224, 224))
img_to_compare = img_to_array(img_to_compare)
img_to_compare = np.expand_dims(img_to_compare, axis=0)
img_to_compare = preprocess_input(img_to_compare)

# Extract feature vector for the image
feature_vec_to_compare = model.predict(img_to_compare)

# Iterate over all images in the folder
import os
folder_path = "/content/data"
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        # Load and preprocess the image
        img = load_img(os.path.join(folder_path, filename), target_size=(224, 224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)

        # Extract feature vector for the image
        feature_vec = model.predict(img)

        # Compute distance between the feature vectors
        dist = distance.euclidean(feature_vec_to_compare, feature_vec)
        if dist == 0:
          print("Existe el producto")
          break
        else:
          print("no tenemos de ese producto")


1/1 [==============================] - 0s 205ms/step
no tenemos de ese producto


In [ ]:
#!pip install numpy
#!pip install scipy
#!pip install tensorflow
#!pip install tensorflow-gpu
#!pip install telegram
#!pip install telebot
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from scipy.spatial import distance

In [ ]:
import os
from telebot import TeleBot

from scipy.spatial import distance
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image

# Load the ResNet50 model
model = ResNet50(weights='imagenet')

def image_comparison(img_path):
    # Load the image you want to compare
    img_to_compare = image.load_img(img_path, target_size=(224, 224))
    img_to_compare = image.img_to_array(img_to_compare)
    img_to_compare = np.expand_dims(img_to_compare, axis=0)
    img_to_compare = preprocess_input(img_to_compare)
    # Extract feature vector for the image
    feature_vec_to_compare = model.predict(img_to_compare)
    # Iterate over all images in the folder
    folder_path = "/content/data"
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            # Load and preprocess the image
            img = image.load_img(os.path.join(folder_path, filename), target_size=(224, 224))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)
            # Extract feature vector for the image
            feature_vec = model.predict(img)
            # Compute distance between the feature vectors
            dist = distance.euclidean(feature_vec_to_compare, feature_vec)
            if dist == 0:
                return "Existe el producto"
                break
            else:
                return "no tenemos de ese producto"

def handle_image(message):
    file_id = message.photo[-1].file_id
    file = bot.get_file(file_id)
    file.download('user_image.jpg')
    result = image_comparison('user_image.jpg')
    bot.send_message(message.chat.id, result)

if __name__ == '__main__':
    # Token del bot
    bot = TeleBot("5776364888:AAFHq7nhyMi7CQwZCWbyWl067d-KIozQB3s")
    bot.message_handler(content_types=["photo"])
      handle_image(message)
    bot.polling()
    bot.infinity_polling()

IndentationError: ignored

In [ ]:
!pip install transformers
!pip install pyTelegramBotAPI
!pip install torch
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install transformers
#!pip install pyTelegramBotAPI
#!pip install torch
#!pip install accelerate

import telebot
import transformers
import torch
import time
import pickle

# Inicializar el bot de Telegram
bot = telebot.TeleBot('5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM')

# Cargar el modelo BERT pre-entrenado
model = transformers.BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Cargar el diccionario de preguntas y respuestas
try:
    with open('qa_dict.pickle', 'rb') as handle:
        qa_dict = pickle.load(handle)
except:
    qa_dict = {}

def select_shipping_plan(message):
    if message.text == 'Envio gratis' or message.text == 'envio gratis':
        bot.reply_to(message, 'Escogiste el plan de envio gratis')
        bot.send_message(message, 'El plan de envio gratis tiene un costo de $0.00 y un tiempo de entrega de 3 a 5 dias habiles')
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
        return
    elif message.text == 'Envio rapido' or message.text == 'envio rapido' or message.text == 'Envio rápido' or message.text == 'envio rápido':
        bot.reply_to(message, 'Escogiste el plan de envio rapido')
        bot.send_message(message, 'El plan de envio rapido tiene un costo de $5.00 y un tiempo de entrega de 1 a 2 dias habiles')
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')

        return
    elif message.text == 'Envio express' or message.text == 'envio express' or message.text == 'Envio expres' or message.text == 'envio expres':
        bot.reply_to(message, 'Escogiste el plan de envio express')
        bot.send_message(message, 'El plan de envio express tiene un costo de $10.00 y un tiempo de entrega de 1 dia habil')
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')

        return

def select_consult(message):
    if message.text == 'Consulta de horarios' or message.text == 'consulta de horarios' or message.text == 'Consulta de horario' or message.text == 'consulta de horario':
        bot.reply_to(message, 'Escogiste la opcion de consultar horarios')
        bot.send_message(message.chat.id, 'Estos son nuestros horarios de atencion:')
        bot.send_photo(message.chat.id, open('imagen1.jpg', 'rb'))
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
        return

    elif message.text == 'Consulta de productos' or message.text == 'consulta de productos' or message.text == 'Consulta de producto' or message.text == 'consulta de producto':
        bot.reply_to(message, 'Escogiste la opcion de consultar productos')
        bot.send_message(message.chat.id, 'Enviame una imagen del producto que deseas consultar')
        # aqui entra el modelo de reconocimiento de imagenes



        return
    elif message.text == 'Destinos de envio' or message.text == 'destinos de envio':
        bot.reply_to(message, 'Escogiste la opcion de consultar destinos de envio')
        bot.send_message(message.chat.id, 'Estos son los destinos de envio:')
        bot.send_photo(message.chat.id, open('imagen2.jpg', 'rb'))
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
        return

def price(message):
    if message.text == 'Cotizacion de envio' or message.text == 'cotizacion de envio' or message.text == 'Cotizacion de envío' or message.text == 'cotizacion de envío' or message.text == 'Cotización de envio' or message.text == 'cotización de envio' or message.text == 'Cotización de envío' or message.text == 'cotización de envío':
        bot.reply_to(message, 'Escogiste la opcion de cotizar envio')
        bot.reply_to(message, 'Para continuar necesito que me proporciones el peso del producto en lbs')
        bot.register_next_step_handler(message, process_peso)
    elif message.text == 'Cotizacion de producto' or message.text == 'cotizacion de producto' or message.text == 'Cotización de producto' or message.text == 'cotización de producto':
        bot.reply_to(message, 'Escogiste la opcion de cotizar producto')
        bot.reply_to(message, 'Por favor, envía el ID del producto o una foto del producto')
        bot.register_next_step_handler(message, process_producto)

import os

class Producto:
    def __init__(self, file_id, precio):
        self.file_id = file_id
        self.precio = precio

def buscar_producto(file_id):
    path_img = os.path.join('bdproductos', file_id + '.jpg')
    path_txt = os.path.join('bdproductos', file_id + '.txt')
    if os.path.exists(path_img) and os.path.exists(path_txt):
        with open(path_txt, 'rb') as f:
            precio = f.read()
        return Producto(file_id, precio)
    return None

def process_producto(message):
    if message.photo:
        file_id = message.photo[-1].file_id
        producto = buscar_producto(file_id)
        if producto:
            bot.send_message(message.chat.id, f'El precio del producto es {producto.precio}')
        else:
            bot.send_message(message.chat.id, 'Producto no encontrado')
    elif message.text:
        producto = buscar_producto(message.text)
        if producto:
            bot.send_message(message.chat.id, f'El precio del producto es {producto.precio}')
        else:
            bot.send_message(message.chat.id, 'Producto no encontrado')



def process_peso(message):
    try:
        peso = float(message.text)
        if peso >= 0 and peso <= 2:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $5')
        elif peso > 2 and peso <= 5:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $10')
        elif peso > 5:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $15')
        else:
            bot.send_message(message.chat.id, 'Peso invalido')
        bot.reply_to(message, "¿Desea seguir cotizando?")
        bot.register_next_step_handler(message, process_continue)
    except ValueError:
        bot.send_message(message.chat.id, 'Peso invalido')

def process_continue(message):
    if message.text.lower() == "si":
        bot.reply_to(message, "¡Genial! Por favor proporciona el peso del producto en lbs")
        bot.register_next_step_handler(message, process_peso)
    elif message.text.lower() == "no":
        bot.reply_to(message, "¡De acuerdo, gracias por usar nuestro servicio de cotización!")
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')

        return
    return

# Manejar mensajes de pregunta de Telegram
@bot.message_handler(func=lambda message: True)
def answer_question(message):

    if message.text == '/start':
        bot.reply_to(message, 'Hola, mi nombre es Serina. Soy un modelo de reconocimiento del lenguaje natural.')
        bot.send_message(message.chat.id, 'Estos son ejemplos de algunas de las opciones que puedes realizar: \n \n /Asesoria \n /Envio \n /Cotizacion \n')
        bot.send_message(message.chat.id, 'Tambien puedes consultar las opciones en el menu de la ezquina inferior izquierda o con el comando /help en todo momento')
        bot.send_message(message.chat.id, 'Sabiendo esto, dime en que te puedo ayudar?')
        return
    
    if message.text == '/asesoria':
        bot.reply_to(message, 'Estoy aqui para orientarte escoge una opcion o dime en que te puedo ayudar: \n \n * Consulta de horarios \n * Consulta de productos \n * Destinos de envio')
        return
    else:
        select_consult(message)
    
    if message.text == '/envio':
        bot.reply_to(message, 'Escoge un plan de envio: \n \n * Envio gratis \n * Envio rapido \n * Envio express')
        return
    else:
        select_shipping_plan(message)

    if message.text == '/cotizacion':
        bot.reply_to(message, 'Escoge una opcion: \n \n * Cotizacion de envio \n * Cotizacion de producto')
        return
    else:
        price(message)
    

    #detener el bot
    if message.text == '/stop':
        bot.reply_to(message, 'Rutinas detenidas')
        return

    if message.text == '/help':
        #muestra la lista de comandos
        bot.reply_to(message, 'Lista de comandos: \n \n /start \n /asesoria \n /envio \n /cotizacion \n /stop \n /restart \n /help')
        return

    #reiiciar el bot
    if message.text == '/restart':
        bot.reply_to(message, 'Rutinas reiniciadas')
        return
    
    
    # Extraer la pregunta del mensaje
    question = message.text

    # Buscar en el diccionario si la pregunta ya ha sido respondida anteriormente
    if question in qa_dict:
        answer = qa_dict[question]
    else:
        #extraer texto de un archivo de texto
        with open('texto.txt', 'r') as file:
            text = file.read().replace('\n', '')
            
        try:
            inputs = tokenizer.encode_plus(question, text, return_tensors='pt', add_special_tokens=True)
            output = model(**inputs)
        except Exception as e:
            bot.reply_to(message, 'Lo siento, ha ocurrido un error al procesar tu pregunta')
            return

        


        # Obtener la respuesta
        answer_start_scores, answer_end_scores = output[:2]
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end],skip_special_tokens=True)

        #excepcion de errores
        if answer == '[CLS]' or answer == '[SEP]':
            answer = 'No se pudo encontrar una respuesta'
        

        # Almacenar la pregunta y la respuesta en el historial
        qa_dict[question] = answer
        with open("qa_dict.pickle", "wb") as f:
            pickle.dump(qa_dict, f)

    # Enviar la respuesta al usuario
    bot.reply_to(message, answer)


# Ejecutar el bot
while True:
    try:
        bot.polling()
    except Exception:
        time.sleep(15)

In [ ]:
#Dependencia e importaciones
!pip install transformers
!pip install pyTelegramBotAPI
!pip install torch
!pip install accelerate
!pip install pyTelegramBotAPI
!pip install reportlab
!pip install python-telegram-bot

from tensorflow.keras.applications.resnet50 import ResNet50
from keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from scipy.spatial import distance
import telebot
import transformers
import torch
import time
import pickle

# Api Key
bot = telebot.TeleBot('5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM')



# Cargar el modelo BERT pre-entrenado
model = transformers.BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Cargar el diccionario de preguntas y respuestas
try:
    with open('qa_dict.pickle', 'rb') as handle:
        qa_dict = pickle.load(handle)
except:
    qa_dict = {}

def select_shipping_plan(message):
    if message.text == 'Envio gratis' or message.text == 'envio gratis':
        bot.reply_to(message, 'Escogiste el plan de envio gratis')
        bot.send_message(message.chat.id, 'Para regresar al menu principal presiona en /start o escibelo')
        return
    elif message.text == 'Envio rapido' or message.text == 'envio rapido':
        bot.reply_to(message, 'Escogiste el plan de envio rapido')
        bot.send_message(message.chat.id, 'Para regresar al menu principal presiona en /start o escibelo')
        return
    elif message.text == 'Envio express' or message.text == 'envio express':
        bot.reply_to(message, 'Escogiste el plan de envio express')
        bot.send_message(message.chat.id, 'Para regresar al menu principal presiona en /start o escibelo')
        return

def select_consult(message):
    if message.text == 'Consulta de horarios' or message.text == 'consulta de horarios':
        bot.reply_to(message, 'Escogiste la opcion de consultar horarios')
        bot.send_message(message.chat.id, 'Estos son nuestros horarios de atencion:')
        bot.send_photo(message.chat.id, open('imagen1.jpg', 'rb'))
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
        return

    elif message.text == 'Consulta de productos' or message.text == 'consulta de productos':
        bot.reply_to(message, 'Escogiste la opcion de consultar productos')
        bot.send_message(message.chat.id, 'Enviame una imagen del producto que deseas consultar')
        # aqui entra el modelo de reconocimiento de imagenes
        @bot.message_handler(content_types=['photo'])
        def recibe_imagen(message):
          handle_image(message)

        model = ResNet50(weights='imagenet')

        def image_comparison(img_path):
          img_to_compare = image.load_img(img_path, target_size=(224, 224))
          img_to_compare = image.img_to_array(img_to_compare)
          img_to_compare = np.expand_dims(img_to_compare, axis=0)
          img_to_compare = preprocess_input(img_to_compare)

          feature_vec_to_compare = model.predict(img_to_compare)
  
          folder_path = "/content/data"
          for filename in os.listdir(folder_path):
            if filename.endswith(".jpg"):
              img = image.load_img(os.path.join(folder_path, filename), target_size=(224, 224))
              img = image.img_to_array(img)
              img = np.expand_dims(img, axis=0)
              img = preprocess_input(img)
          

            if dist == 0:
                bot.send_message(message.chat.id, 'El producto es: ' + filename)
                bot.send_message(message.chat.id, 'El producto se encuentra disponible')
                break
            else:
                bot.send_message(message.chat.id, 'No existe el producto')
                bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
                return


        def handle_image(message):
          file_id = message.photo[-1].file_id
          file = bot.get_file(file_id)
          downloaded = bot.download_file(file.file_path)
          with open ("user_image.jpg","wb") as guardar:
            guardar.write(downloaded)
          result = image_comparison('user_image.jpg')
          bot.send_message(message.chat.id, result)
        return

    elif message.text == 'Destinos de envio' or message.text == 'destinos de envio':
        bot.reply_to(message, 'Escogiste la opcion de consultar destinos de envio')
        bot.send_message(message.chat.id, 'Estos son los destinos de envio:')
        bot.send_photo(message.chat.id, open('imagen2.jpg', 'rb'))
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
        return

def price(message):
    if message.text == 'Cotizacion de envio' or message.text == 'cotizacion de envio':
        bot.reply_to(message, 'Escogiste la opcion de cotizar envio')
        bot.reply_to(message, 'Para continuar necesito que me proporciones el peso del producto en lbs')
        bot.register_next_step_handler(message, process_peso)
    elif message.text == 'Cotizacion de producto' or message.text == 'cotizacion de producto':
        bot.reply_to(message, 'Escogiste la opcion de cotizar producto')
        bot.reply_to(message, 'Por favor, envía el ID del producto o una foto del producto')
        bot.register_next_step_handler(message, process_producto)

import os

class Producto:
    def __init__(self, file_id, precio):
        self.file_id = file_id
        self.precio = precio

def buscar_producto(file_id):
    path_img = os.path.join('bdproductos', file_id + '.jpg')
    path_txt = os.path.join('bdproductos', file_id + '.txt')
    if os.path.exists(path_img) and os.path.exists(path_txt):
        with open(path_txt, 'rb') as f:
            precio = f.read()
        return Producto(file_id, precio)
    return None

def process_producto(message):
    if message.photo:
        file_id = message.photo[-1].file_id
        producto = buscar_producto(file_id)
        if producto:
            bot.send_message(message.chat.id, f'El precio del producto es {producto.precio}')
        else:
            bot.send_message(message.chat.id, 'Producto no encontrado')
    elif message.text:
        producto = buscar_producto(message.text)
        if producto:
            bot.send_message(message.chat.id, f'El precio del producto es {producto.precio}')
        else:
            bot.send_message(message.chat.id, 'Producto no encontrado')



def process_peso(message):
    try:
        peso = float(message.text)
        if peso >= 0 and peso <= 2:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $5')
        elif peso > 2 and peso <= 5:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $10')
        elif peso > 5:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $15')
        else:
            bot.send_message(message.chat.id, 'Peso invalido')
        bot.reply_to(message, "¿Desea seguir cotizando?")
        bot.register_next_step_handler(message, process_continue)
    except ValueError:
        bot.send_message(message.chat.id, 'Peso invalido')

def process_continue(message):
    if message.text.lower() == "si":
        bot.reply_to(message, "¡Genial! Por favor proporciona el peso del producto en lbs")
        bot.register_next_step_handler(message, process_peso)
    elif message.text.lower() == "no":
        bot.reply_to(message, "¡De acuerdo, gracias por usar nuestro servicio de cotización!")
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')

        return
    return

# Manejar mensajes de pregunta de Telegram
@bot.message_handler(func=lambda message: True)
def answer_question(message):

    if message.text == '/start':
        bot.reply_to(message, 'Hola, mi nombre es Serina, te estaré ayudando en tus consultas.')
        bot.send_message(message.chat.id, 'Estos son ejemplos de algunas de las opciones que puedes realizar: \n \n /asesoria \n /envio \n /cotizacion \n')
        bot.send_message(message.chat.id, 'Tambien puedes consultar las opciones en el menu de la ezquina inferior izquierda o con el comando /help en todo momento')
        bot.send_message(message.chat.id, 'Sabiendo esto, dime en que te puedo ayudar?')
        return
    
    if message.text == '/asesoria':
        bot.reply_to(message, 'Estoy aqui para orientarte escoge una opcion o dime en que te puedo ayudar: \n \n * Consulta de horarios \n * Consulta de productos \n * Destinos de envio')
        return
    else:
        select_consult(message)
    
    if message.text == '/envio':
        bot.reply_to(message, 'Escoge un plan de envio: \n \n * Envio gratis \n * Envio rapido \n * Envio express')
        return
    else:
        select_shipping_plan(message)

    if message.text == '/cotizacion':
        bot.reply_to(message, 'Escoge una opcion: \n \n * Cotizacion de envio \n * Cotizacion de producto')
        return
    else:
        price(message)
    

    #detener el bot
    if message.text == '/stop':
        bot.reply_to(message, 'Rutinas detenidas')
        return

    if message.text == '/help':
        #muestra la lista de comandos
        bot.reply_to(message, 'Lista de comandos: \n \n /start \n /Asesoria \n /Envio \n /Cotizacion \n /stop \n /restart \n /help')
        return

    #reiiciar el bot
    if message.text == '/restart':
        bot.reply_to(message, 'Rutinas reiniciadas')
        return
    
    
    # Extraer la pregunta del mensaje
    question = message.text

    # Buscar en el diccionario si la pregunta ya ha sido respondida anteriormente
    if question in qa_dict:
        answer = qa_dict[question]
    else:
        #extraer texto de un archivo de texto
        with open('texto.txt', 'r') as file:
            text = file.read().replace('\n', '')
            
        try:
            inputs = tokenizer.encode_plus(question, text, return_tensors='pt', add_special_tokens=True)
            output = model(**inputs)
        except Exception as e:
            bot.reply_to(message, 'Lo siento, ha ocurrido un error al procesar tu pregunta')
            return

        


        # Obtener la respuesta
        answer_start_scores, answer_end_scores = output[:2]
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end],skip_special_tokens=True)

        #excepcion de errores
        if answer == '[CLS]' or answer == '[SEP]':
            answer = 'No se pudo encontrar una respuesta'
        

        # Almacenar la pregunta y la respuesta en el historial
        qa_dict[question] = answer
        with open("qa_dict.pickle", "wb") as f:
            pickle.dump(qa_dict, f)

    # Enviar la respuesta al usuario
    bot.reply_to(message, answer)


# Ejecutar el bot
#while True:
   # try:
    #    bot.polling()
    #except Exception:  
     #   time.sleep(1)

bot.polling()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
1/1 [==============================] - 1s 789ms/step


UnboundLocalError: ignored

In [ ]:
#Dependencia e importaciones
!pip install transformers
!pip install pyTelegramBotAPI
!pip install torch
!pip install accelerate
!pip install pyTelegramBotAPI
!pip install reportlab
!pip install python-telegram-bot
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from scipy.spatial import distance
import datetime

#---------------------------------------------------------------------------------------------------------------------------
#!pip install transformers
#!pip install pyTelegramBotAPI
#!pip install torch
#!pip install accelerate

import telebot
import transformers
import torch
import time
import pickle

# Inicializar el bot de Telegram
#api key es de fermin
bot = telebot.TeleBot('5776364888:AAFHq7nhyMi7CQwZCWbyWl067d-KIozQB3s')
#api key de pedro: 5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM
#api key de fermin: 5776364888:AAFHq7nhyMi7CQwZCWbyWl067d-KIozQB3s


# Cargar el modelo BERT pre-entrenado
model = transformers.BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Cargar el diccionario de preguntas y respuestas
try:
    with open('qa_dict.pickle', 'rb') as handle:
        qa_dict = pickle.load(handle)
except:
    qa_dict = {}

def select_shipping_plan(message):
    if message.text == 'Envio gratis' or message.text == 'envio gratis':
        bot.reply_to(message, 'Escogiste el plan de envio gratis')
        bot.send_message(message.chat.id, 'Para regresar al menu principal presiona en /start o escibelo')
        return
    elif message.text == 'Envio rapido' or message.text == 'envio rapido':
        bot.reply_to(message, 'Escogiste el plan de envio rapido')
        bot.send_message(message.chat.id, 'Para regresar al menu principal presiona en /start o escibelo')
        return
    elif message.text == 'Envio express' or message.text == 'envio express':
        bot.reply_to(message, 'Escogiste el plan de envio express')
        bot.send_message(message.chat.id, 'Para regresar al menu principal presiona en /start o escibelo')
        return

def select_consult(message):
    if message.text == 'Consulta de horarios' or message.text == 'consulta de horarios':
        bot.reply_to(message, 'Escogiste la opcion de consultar horarios')
        bot.send_message(message.chat.id, 'Estos son nuestros horarios de atencion:')
        bot.send_photo(message.chat.id, open('imagen1.jpg', 'rb'))
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
        return

    elif message.text == 'Consulta de productos' or message.text == 'consulta de productos':
        bot.reply_to(message, 'Escogiste la opcion de consultar productos')
        bot.send_message(message.chat.id, 'Enviame una imagen del producto que deseas consultar')
        # aqui entra el modelo de reconocimiento de imagenes
        @bot.message_handler(content_types=['photo'])
        def recibe_imagen(message):
          handle_image(message)

        model = ResNet50(weights='imagenet')

        def image_comparison(img_path):
          img_to_compare = image.load_img(img_path, target_size=(224, 224))
          img_to_compare = image.img_to_array(img_to_compare)
          img_to_compare = np.expand_dims(img_to_compare, axis=0)
          img_to_compare = preprocess_input(img_to_compare)

          feature_vec_to_compare = model.predict(img_to_compare)
  
          folder_path = "/content/content/data/"
          for filename in os.listdir(folder_path):
            if filename.endswith(".jpg"):
              img = image.load_img(os.path.join(folder_path, filename), target_size=(224, 224))
              img = image.img_to_array(img)
              img = np.expand_dims(img, axis=0)
              img = preprocess_input(img)
          
            feature_vec = model.predict(img)
           
            dist = distance.euclidean(feature_vec_to_compare, feature_vec)
            if dist == 0:
                return "Existe el producto"
                break
            else:
                return "No contamos con este producto en nuestro stock, estaremos avisando en nuestra pagina web, si requieres regresar al menu presiona en /start"

        def handle_image(message):
          file_id = message.photo[-1].file_id
          file = bot.get_file(file_id)
          downloaded = bot.download_file(file.file_path)
          with open ("user_image.jpg","wb") as guardar:
            guardar.write(downloaded)
          result = image_comparison('user_image.jpg')
          bot.send_message(message.chat.id, result)

    elif message.text == 'Destinos de envio' or message.text == 'destinos de envio':
        bot.reply_to(message, 'Escogiste la opcion de consultar destinos de envio')
        bot.send_message(message.chat.id, 'Estos son los destinos de envio:')
        bot.send_photo(message.chat.id, open('imagen2.jpg', 'rb'))
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')
        return

def price(message):
    if message.text == 'Cotizacion de envio' or message.text == 'cotizacion de envio':
        bot.reply_to(message, 'Escogiste la opcion de cotizar envio')
        bot.reply_to(message, 'Para continuar necesito que me proporciones el peso del producto en lbs')
        bot.register_next_step_handler(message, process_peso)
    elif message.text == 'Cotizacion de producto' or message.text == 'cotizacion de producto':
        bot.reply_to(message, 'Escogiste la opcion de cotizar producto')
        bot.reply_to(message, 'Por favor, envía el ID del producto o una foto del producto')
        bot.register_next_step_handler(message, process_producto)

import os

class Producto:
    def __init__(self, file_id, precio):
        self.file_id = file_id
        self.precio = precio

def buscar_producto(file_id):
    path_img = os.path.join('bdproductos', file_id + '.jpg')
    path_txt = os.path.join('bdproductos', file_id + '.txt')
    if os.path.exists(path_img) and os.path.exists(path_txt):
        with open(path_txt, 'rb') as f:
            precio = f.read()
        return Producto(file_id, precio)
    return None

def process_producto(message):
    if message.photo:
        file_id = message.photo[-1].file_id
        producto = buscar_producto(file_id)
        if producto:
            bot.send_message(message.chat.id, f'El precio del producto es {producto.precio}')
        else:
            bot.send_message(message.chat.id, 'Producto no encontrado')
    elif message.text:
        producto = buscar_producto(message.text)
        if producto:
            bot.send_message(message.chat.id, f'El precio del producto es {producto.precio}')
        else:
            bot.send_message(message.chat.id, 'Producto no encontrado')



def process_peso(message):
    try:
        peso = float(message.text)
        if peso >= 0 and peso <= 2:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $5')
        elif peso > 2 and peso <= 5:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $10')
        elif peso > 5:
            bot.send_message(message.chat.id, 'El precio de envío para ese peso es $15')
        else:
            bot.send_message(message.chat.id, 'Peso invalido')
        bot.reply_to(message, "¿Desea seguir cotizando?")
        bot.register_next_step_handler(message, process_continue)
    except ValueError:
        bot.send_message(message.chat.id, 'Peso invalido')

def process_continue(message):
    if message.text.lower() == "si":
        bot.reply_to(message, "¡Genial! Por favor proporciona el peso del producto en lbs")
        bot.register_next_step_handler(message, process_peso)
    elif message.text.lower() == "no":
        bot.reply_to(message, "¡De acuerdo, gracias por usar nuestro servicio de cotización!")
        bot.send_message(message.chat.id, 'Presiona aqui para volver al menu principal: \n \n /start')

        return
    return

# Manejar mensajes de pregunta de Telegram
@bot.message_handler(func=lambda message: True)
def answer_question(message):

    if message.text == '/start':
        bot.reply_to(message, 'Hola, mi nombre es Serina, te estaré ayudando en tus consultas.')
        bot.send_message(message.chat.id, 'Estos son ejemplos de algunas de las opciones que puedes realizar: \n \n /asesoria \n /envio \n /cotizacion \n')
        bot.send_message(message.chat.id, 'Tambien puedes consultar las opciones en el menu de la ezquina inferior izquierda o con el comando /help en todo momento')
        bot.send_message(message.chat.id, 'Sabiendo esto, dime en que te puedo ayudar?')
        return
    
    if message.text == '/asesoria':
        bot.reply_to(message, 'Estoy aqui para orientarte escoge una opcion o dime en que te puedo ayudar: \n \n * Consulta de horarios \n * Consulta de productos \n * Destinos de envio')
        return
    else:
        select_consult(message)
    
    if message.text == '/envio':
        bot.reply_to(message, 'Escoge un plan de envio: \n \n * Envio gratis \n * Envio rapido \n * Envio express')
        return
    else:
        select_shipping_plan(message)

    if message.text == '/cotizacion':
        bot.reply_to(message, 'Escoge una opcion: \n \n * Cotizacion de envio \n * Cotizacion de producto')
        return
    else:
        price(message)
    

    #detener el bot
    if message.text == '/stop':
        bot.reply_to(message, 'Rutinas detenidas')
        return

    if message.text == '/help':
        #muestra la lista de comandos
        bot.reply_to(message, 'Lista de comandos: \n \n /start \n /Asesoria \n /Envio \n /Cotizacion \n /stop \n /restart \n /help')
        return

    #reiiciar el bot
    if message.text == '/restart':
        bot.reply_to(message, 'Rutinas reiniciadas')
        return
    
    
    # Extraer la pregunta del mensaje
    question = message.text

    # Buscar en el diccionario si la pregunta ya ha sido respondida anteriormente
    if question in qa_dict:
        answer = qa_dict[question]
    else:
        #extraer texto de un archivo de texto
        with open('texto.txt', 'r') as file:
            text = file.read().replace('\n', '')
            
        try:
            inputs = tokenizer.encode_plus(question, text, return_tensors='pt', add_special_tokens=True)
            output = model(**inputs)
        except Exception as e:
            bot.reply_to(message, 'Lo siento, ha ocurrido un error al procesar tu pregunta')
            return

        


        # Obtener la respuesta
        answer_start_scores, answer_end_scores = output[:2]
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end],skip_special_tokens=True)

        #excepcion de errores
        if answer == '[CLS]' or answer == '[SEP]':
            answer = 'No se pudo encontrar una respuesta'
        

        # Almacenar la pregunta y la respuesta en el historial
        qa_dict[question] = answer
        with open("qa_dict.pickle", "wb") as f:
            pickle.dump(qa_dict, f)

    # Enviar la respuesta al usuario
    bot.reply_to(message, answer)


# Ejecutar el bot
while True:
    try:
        bot.polling()
    except Exception:
        time.sleep(15)